In [84]:
from operator import neg
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict
import json
import time

In [37]:
client_id = ""
client_secret = ""

In [42]:
# Get a bearer token.
petfinder_token_url = "https://api.petfinder.com/v2/oauth2/token"

data = {'grant_type' : 'client_credentials', 'client_id' : client_id , 'client_secret' : client_secret}

response = requests.post(petfinder_token_url, data = data)

json_response = json.loads(response.content)

bearer_token = json_response['access_token']

In [53]:
# Get Best Friends Society Organization.
petfinder_get_organizations_url = "https://api.petfinder.com/v2/organizations"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

params = {'name' : 'Best Friends Animal Society'}

response = requests.get(petfinder_get_organizations_url, params = params, headers = headers)

json_response = json.loads(response.content)

for i in range(5):
    print(json_response['organizations'][i])

{'id': 'UT81', 'name': 'Best Friends Animal Society-Utah', 'email': 'utahadoptions@bestfriends.org', 'phone': '(801) 574-2454', 'address': {'address1': '2005 S 1100 E', 'address2': None, 'city': 'Salt Lake City', 'state': 'UT', 'postcode': '84106', 'country': 'US'}, 'hours': {'monday': '10 am - 7 pm', 'tuesday': '10 am - 7 pm', 'wednesday': '10 am - 7 pm', 'thursday': '10 am - 7 pm', 'friday': '10 am - 7 pm', 'saturday': '10 am - 7 pm', 'sunday': '10 am - 7 pm'}, 'url': 'https://www.petfinder.com/member/us/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'website': 'http://utah.bestfriends.org', 'mission_statement': 'The mission of Best Friends Animal Society is to bring about a time when there are No More Homeless Pets....', 'adoption': {'policy': 'Due to Covid-19, our SLC Lifesaving Center is temporarily closed. All pets are currently in foster homes and adoption meet...', 'url': 'https://utah.bestfriends.org/get-involved/ado

In [58]:
# The Best Friends Animal Society's main office is the one with id "UT77".
# Getting some profiles from that organization.
petfinder_get_animals_url = "https://api.petfinder.com/v2/animals"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

organization_id = "UT77"
limit = 100

params = {'organization' : organization_id, 'limit' : limit}

response = requests.get(petfinder_get_animals_url, params = params, headers = headers)

json_response = json.loads(response.content)

print(json_response)

{'animals': [{'id': 58058116, 'organization_id': 'UT77', 'url': 'https://www.petfinder.com/dog/piper-mclean-58058116/ut/kanab/best-friends-animal-society-ut77/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'type': 'Dog', 'species': 'Dog', 'breeds': {'primary': 'Border Collie', 'secondary': None, 'mixed': True, 'unknown': False}, 'colors': {'primary': 'Black', 'secondary': 'White / Cream', 'tertiary': None}, 'age': 'Young', 'gender': 'Female', 'size': 'Medium', 'coat': None, 'attributes': {'spayed_neutered': False, 'house_trained': False, 'declawed': None, 'special_needs': False, 'shots_current': True}, 'environment': {'children': None, 'dogs': None, 'cats': None}, 'tags': [], 'name': 'Piper McLean', 'description': None, 'organization_animal_id': 'BKNB-A-40749', 'photos': [{'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/58058116/1/?bust=1665431770&width=100', 'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/58058116/1/?bust=1665431770&width=300', 'large': 'ht

In [59]:
print(len(json_response['animals']))

100


In [81]:
# Harmonize data.
animals = json_response['animals']
harmonized_animals = []

for animal in animals:

    harmonized_animal = {'Type' : animal['type'], 'Name' : animal['name'], 'Age' : animal['age'],\
                           'Breed1' : animal['breeds']['primary'], 'Breed2' : animal['breeds']['secondary'], \
                           'Gender' : animal['gender'], 'Color1' : animal['colors']['primary'], \
                           'Color2' : animal['colors']['secondary'], 'Color3' : animal['colors']['tertiary'], \
                           'MaturitySize' : animal['size'], 'FurLength' : None, 'Vaccinated' : animal['attributes']['shots_current'], \
                           'Dewormed' : None, 'Sterilized' : animal['attributes']['spayed_neutered'], 'Health' : None, \
                           'Quantity' : 1, 'Fee' : None, 'State' : animal['contact']['address']['state'], 'RescuerID' : None, \
                           'VideoAmt' : len(animal['videos']), 'Description' : animal['description'], 'PetID' : animal['id'], \
                           'PhotoAmt' : len(animal['photos']), 'AdoptionSpeed' : None, 'id' : animal['id'] }
    harmonized_animals.append(harmonized_animal)

print(harmonized_animals)

[{'Type': 'Dog', 'Name': 'Piper McLean', 'Age': 'Young', 'Breed1': 'Border Collie', 'Breed2': None, 'Gender': 'Female', 'Color1': 'Black', 'Color2': 'White / Cream', 'Color3': None, 'MaturitySize': 'Medium', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': False, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 58058116, 'PhotoAmt': 1, 'AdoptionSpeed': None, 'id': 58058116}, {'Type': 'Dog', 'Name': 'Brew', 'Age': 'Senior', 'Breed1': 'American Bulldog', 'Breed2': None, 'Gender': 'Male', 'Color1': 'White / Cream', 'Color2': 'Black', 'Color3': None, 'MaturitySize': 'Large', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': True, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': '&amp;#34;Hello, hi, will you be my friend? My foster mom calls me a gentle goofy giant. I&amp;#39;m happy doing just...', 'PetID': 58054880,

In [86]:
df = pd.DataFrame(harmonized_animals)

In [87]:
df.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,id
0,Dog,Piper McLean,Young,Border Collie,None,Female,Black,White / Cream,None,Medium,...,1,None,UT,None,0,None,58058116,1,None,58058116
1,Dog,Brew,Senior,American Bulldog,None,Male,White / Cream,Black,None,Large,...,1,None,UT,None,0,"&amp;#34;Hello, hi, will you be my friend? My ...",58054880,6,None,58054880
2,Dog,Pye,Adult,Pit Bull Terrier,None,Female,Gray / Blue / Silver,None,None,Large,...,1,None,UT,None,0,&amp;#34;I might look a bit concerned in my ph...,58054879,6,None,58054879
3,Dog,Meyer,Adult,Australian Cattle Dog / Blue Heeler,None,Male,Gray / Blue / Silver,None,None,Medium,...,1,None,UT,None,0,Meyer is a sweet guy who is learning the routi...,58049180,1,None,58049180
4,Dog,Duchess,Young,Shepherd,None,Female,Black,Brown / Chocolate,None,Medium,...,1,None,UT,None,0,None,58047970,0,None,58047970
5,Dog,Rosie,Adult,Pit Bull Terrier,None,Female,Brown / Chocolate,White / Cream,None,Medium,...,1,None,UT,None,0,None,58047969,6,None,58047969
6,Cat,Chastain,Baby,Domestic Long Hair,None,Female,Gray / Blue / Silver,None,None,Small,...,1,None,UT,None,0,None,58045935,0,None,58045935
7,Rabbit,Sasquatch,Young,Lionhead,None,Male,None,None,None,Medium,...,1,None,UT,None,0,None,58037978,1,None,58037978
8,Cat,Cresseida,Adult,Domestic Short Hair,None,Female,Black,None,None,Small,...,1,None,UT,None,0,None,58025511,1,None,58025511
9,Dog,Driz&#39;zt,Senior,German Shepherd Dog,None,Female,Black,Brown / Chocolate,None,Large,...,1,None,UT,None,0,None,58016299,1,None,58016299


In [90]:
df.to_csv('data/best_friends_animal_society/test.csv', index = False)